# Shapes

In [7]:
#!pip install sodapy

In [8]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata

In [9]:
#!pip install geopy

In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Guatemala'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Guatemala are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Guatemala are 15.6356088, -89.8988087.


In [11]:
#!pip install pyshp==1.1.7

In [12]:
shp_path = "shapes\gadm36_GTM_2.shp"

import shapefile
from json import dumps

# read the shapefile
reader = shapefile.Reader(shp_path)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(buffer)

In [13]:
df.head()

,geometry,properties,type
0,"{'type': 'Polygon', 'coordinates': (((-89.4778...","{'GID_0': 'GTM', 'NAME_0': 'Guatemala', 'GID_1...",Feature
1,"{'type': 'Polygon', 'coordinates': (((-89.9826...","{'GID_0': 'GTM', 'NAME_0': 'Guatemala', 'GID_1...",Feature
2,"{'type': 'Polygon', 'coordinates': (((-90.3900...","{'GID_0': 'GTM', 'NAME_0': 'Guatemala', 'GID_1...",Feature
3,"{'type': 'Polygon', 'coordinates': (((-89.7121...","{'GID_0': 'GTM', 'NAME_0': 'Guatemala', 'GID_1...",Feature
4,"{'type': 'Polygon', 'coordinates': (((-89.9954...","{'GID_0': 'GTM', 'NAME_0': 'Guatemala', 'GID_1...",Feature


In [14]:
def getPropiedad(propiedad, valor):    
    return valor.replace("'","").split(propiedad)[1].split(",")[0].strip()   

In [15]:
df['Departamento'] = df.properties.map(lambda x: getPropiedad("NAME_0:",str(x)))

In [16]:
df['Municipio'] = df.properties.map(lambda x: getPropiedad("NAME_1:",str(x)))

In [17]:
df['Zona'] = df.properties.map(lambda x: getPropiedad("NAME_2:",str(x)))

In [18]:
df.drop(['properties','type'], inplace=True, axis=1)

In [19]:
df.head()

,geometry,Departamento,Municipio,Zona
0,"{'type': 'Polygon', 'coordinates': (((-89.4778...",Guatemala,Alta Verapaz,Chahal
1,"{'type': 'Polygon', 'coordinates': (((-89.9826...",Guatemala,Alta Verapaz,Chisec
2,"{'type': 'Polygon', 'coordinates': (((-90.3900...",Guatemala,Alta Verapaz,Cobán
3,"{'type': 'Polygon', 'coordinates': (((-89.7121...",Guatemala,Alta Verapaz,Fray Bartolomé de las Casas
4,"{'type': 'Polygon', 'coordinates': (((-89.9954...",Guatemala,Alta Verapaz,Lanquín


In [20]:
features = []
#df = df[df['Municipio']=='Guatemala']
for muni,zona,geometry in zip(df['Municipio'], df['Zona'], df['geometry']):
    properties = {
        'type': 'Feature',
        'properties': {'name':(muni.upper()+','+zona.upper())},'geometry':geometry
    }
    features.append(properties)    
    
geofeatures = {    
    'features': features,
    'type': 'FeatureCollection'
}

geo_str = dumps(geofeatures) 

In [21]:
df.head()

,geometry,Departamento,Municipio,Zona
0,"{'type': 'Polygon', 'coordinates': (((-89.4778...",Guatemala,Alta Verapaz,Chahal
1,"{'type': 'Polygon', 'coordinates': (((-89.9826...",Guatemala,Alta Verapaz,Chisec
2,"{'type': 'Polygon', 'coordinates': (((-90.3900...",Guatemala,Alta Verapaz,Cobán
3,"{'type': 'Polygon', 'coordinates': (((-89.7121...",Guatemala,Alta Verapaz,Fray Bartolomé de las Casas
4,"{'type': 'Polygon', 'coordinates': (((-89.9954...",Guatemala,Alta Verapaz,Lanquín


In [22]:
import folium # map rendering library

# create map of Baltimore using latitude and longitude values
map_gt = folium.Map(location=[latitude, longitude], zoom_start=8)

# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
map_gt.choropleth(
    geo_data=geo_str,    
    fill_color='YlOrRd', 
    fill_opacity=0.4, 
    line_opacity=0.9,
    legend_name=''
)    

map_gt